In [ ]:
# !pip install numpy pandas scipy 

In [ ]:
import numpy as np
from scipy.linalg import lstsq
import random
import pandas as pd
import os

type_converter = {i: lambda x: int(x, 16) for i in range(16)}
LAB_PATH = "/scratch/net4/HOS/Traces"
ATTACK_PATH = "/home/navanerj/Downloads/ATTACK"
MAX_ROWS = 30000 #Total Number of rows to be selected
ROUND1_START = 20000
ROUND1_END = 25000
SCALE = 3
m_off = 0
AES_SBOX = [0x6, 0xB, 0x5, 0x4, 0x2, 0xE, 0x7, 0xA,0x9, 0xD, 0xF, 0xC, 0x3, 0x1, 0x0, 0x8]
random_plaintexts = np.loadtxt(os.path.join(LAB_PATH,"plaintexts.txt"), dtype=np.uint8, max_rows=MAX_ROWS, converters=type_converter)
random_ciphertexts = np.loadtxt(os.path.join(LAB_PATH,"ciphertexts.txt"), dtype=np.uint8, max_rows=MAX_ROWS, converters=type_converter)
random_traces = np.loadtxt(os.path.join(LAB_PATH,"traces.txt"), max_rows=MAX_ROWS)
unscaled_traces = random_traces[:,ROUND1_START:ROUND1_END]

scaled_traces = ((unscaled_traces / 127) * 4) * SCALE + m_off

print("unscaled traces shape: ", unscaled_traces.shape)
print("scaled traces shape: ", scaled_traces.shape)

## Stochastic Model for DSCA 
- Constructing Basis functions
- Profiling based on SBOX output of 1st round
- Key Extraction based on minimum residual error

In [ ]:
import numpy as np
from scipy.linalg import lstsq, pinv
import random
from numpy import linalg

class BasisFunctions:
    """Handles the precomputation of the basis matrix G for a 4-bit S-Box."""
    def __init__(self, sbox, basis_type='bits'):
        self.sbox = sbox
        if basis_type in ["hw", "lsb"]:
            self.num_basis = 2
        else:
            self.num_basis = 5
        self.basis_type = basis_type
        self.G = self._build_basis_matrix()

    def _build_basis_matrix(self):
        """Precomputes the basis matrix G for all phi values (0-15) based on actual bit value and HW"""
        G = np.zeros((16, self.num_basis), dtype=float)
        for phi in range(16):
            sbox_output = self.sbox[phi]
            if(self.basis_type == 'bits'):
                G[phi] = [1.0]+[float(i) for i in format(sbox_output, '04b')]
            elif(self.basis_type == 'hw_bits'):
                G[phi] = [1.0] + [float(i) for i in format(bin(sbox_output).count('1'),'04b')]
            elif(self.basis_type == 'hw'):
                G[phi] = [1.0] + [float(bin(sbox_output).count('1'))]
            elif(self.basis_type == 'lsb'):
                G[phi] = [1.0, float(sbox_output & 1)]
        return G

class Profiling:
    """Manages the profiling phase: beta estimation, time point selection, and covariance estimation."""
    def __init__(self, basis_functions):
        self.basis_functions = basis_functions
        self.betas = None
        self.ts = None
        self.cov = None

    def estimate_betas(self, traces_prof, x_prof, k_b, start_time, end_time):
        phi_prof = np.bitwise_xor(x_prof, k_b) & 0x0F
        G_prof = self.basis_functions.G[phi_prof]  # shape (N1, num_basis)
        I = traces_prof[:, start_time:end_time]  # shape (N1, segment_length)
        G_pinv = pinv(G_prof)
        betas = G_pinv @ I    # Shape (num_basis, segment_length)
        self.betas = betas.T  # Match old shape (segment_length, num_basis)
        self.start_time = start_time
        return self.betas

    def select_time_points(self,betas, tau,N = 15, selection_mode='S2', traces_prof=None):
        """Selects relevant time points ts based on the norm of data-dependent betas."""
        norm_b = np.linalg.norm(betas[:, 1:], axis=1)  # Exclude constant term
        if selection_mode == 'S3':
            self.ts = [np.argmax(norm_b)]  #Single max peak
        elif selection_mode == 'S2':
            self.ts = [int(x) for x in np.argsort(norm_b)[-N:]]  # Top N peaks
        elif selection_mode == 'S1':
            self.ts = np.where(norm_b >= tau)[0]  #Threshold-based
        elif selection_mode in ['S4', 'S5']:
            if traces_prof is None:
                raise ValueError("traces_prof required for S4/S5 selection modes")
            var_t = np.var(traces_prof, axis=0)
            mask = (norm_b >= tau) & (norm_b > var_t)
            self.ts = np.where(mask)[0]
            if selection_mode == 'S5':
                extra_mask = (norm_b >= tau / 2) & ~mask
                self.ts = np.concatenate((self.ts, np.where(extra_mask)[0]))
        else:
            raise ValueError(f"Unknown selection_mode: {selection_mode}")
        return self.ts
    
    def estimate_covariance(self, traces_noise, x_noise, k_b):
        """Estimates the covariance matrix of residuals for ML key extraction."""
        N2 = len(x_noise)
        m = len(self.ts)
        phi_noise = np.bitwise_xor(x_noise, k_b) & 0x0F
        G_phi = self.basis_functions.G[phi_noise]  # [N2, 5]
        betas_ts = self.betas[self.ts]  # [m, 5]
        h_star_all = np.einsum('ij,kj->ik', G_phi, betas_ts)  # [N2, m]
        residuals = traces_noise[:, self.ts] - h_star_all
        self.cov = np.cov(residuals.T)

class KeyExtraction:
    """Handles key extraction using minimum or maximum likelihood principles."""
    def __init__(self, profiling):
        self.profiling = profiling
        self.basis_functions = profiling.basis_functions
        self.betas = profiling.betas
        self.ts = profiling.ts
        self.cov = profiling.cov

    def extract_key(self, traces_attack, x_attack,betas, ts,  N3, method='minimum'):
        """Extracts the subkey using the specified method."""
        extracted_key = {i: 0 for i in range(16)}
        num_candidates = 16
        key_scores = np.zeros(num_candidates)
        G = self.basis_functions.G
        betas_ts = betas[ts]
        if method == 'minimum':
            min_diff = float('inf')
            best_k = None
            for k_prime in range(num_candidates):
                diff = 0.0
                for j in range(N3):
                    phi_j = np.bitwise_xor(x_attack[j], k_prime) & 0x0F
                    h_j = betas_ts @ G[phi_j]
                    i_j = traces_attack[j, ts]
                    diff += np.sum((i_j - h_j) ** 2)
                avg_diff = diff / N3
                if avg_diff < min_diff:
                    min_diff = avg_diff
                    best_k = k_prime
                    extracted_key[k_prime] = extracted_key[best_k] + 1
            return best_k, extracted_key
        
        elif method == 'maximum':
            if self.cov is None:
                raise ValueError("Covariance matrix not available. Estimate it in profiling first.")

            # Precompute covariance inverse and determinant for Gaussian PDF
            try:
                C_inv = np.linalg.inv(self.cov)
            except np.linalg.LinAlgError:
                C_inv = np.linalg.pinv(self.cov)
            best_k = None
            for k_prime in range(num_candidates):
                val = 0.0
                for j in range(N3):
                    phi_j = np.bitwise_xor(x_attack[j], k_prime) & 0x0F
                    h_j = betas_ts @ G[phi_j]     # Predicted deterministic leakage
                    i_j = traces_attack[j, ts]    # Measured leakage
                    delta = i_j - h_j             # Residual noise vector
                    val += delta.T @ C_inv @ delta
                key_scores[k_prime] = val

            top_k_indices = np.argsort(key_scores)[:5]
            top_k_scores = key_scores[top_k_indices]
            top_candidates = {int(k): float(s) for k, s in zip(top_k_indices, top_k_scores)}
            return int(top_k_indices[0]), top_candidates

        else:
            raise ValueError(f"Unknown method: {method}")
    
def create_range(start_time=0, end_time=5000, increments=(5, 10), widths=(10, 20)):
    ranges = set()
    widths = tuple(sorted(widths))
    min_w = widths[0]

    for inc in increments:
        s = start_time
        while s + min_w <= end_time:
            for w in widths:
                end = s + w
                if end > end_time:
                    break
                ranges.add((s, end))
            s += inc
    return sorted(ranges)

## Profiling and Key Extraction using Known Key

In [42]:
# import SDSCA as sd
total_prof = 10000
key = np.array([15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0], dtype=np.uint8)
ATTACK_START = 20000
ATTACK_END = 23000
TOTAL_ATTACK = ATTACK_END - ATTACK_START
PROF_START = 0
PROF_END = 5000
correct_count = 0
top_n = 4
results = []
METHOD = "maximum"
N_POINTS = 20
N2 = 1000
BASIS_TYPE = "hw"
for byte_idx in range(16):
    plaintexts_all = random_plaintexts[:, byte_idx] & 0x0F  # Lower 4 bits
    k_b = key[byte_idx] & 0x0F
    traces_prof = scaled_traces[:total_prof]
    plaintexts_prof = plaintexts_all[:total_prof]
    AES_SBOX = [0x6, 0xB, 0x5, 0x4, 0x2, 0xE, 0x7, 0xA,0x9, 0xD, 0xF, 0xC, 0x3, 0x1, 0x0, 0x8]
    bf = BasisFunctions(AES_SBOX, basis_type=BASIS_TYPE)
    prof = Profiling(bf)
    betas = prof.estimate_betas(traces_prof, plaintexts_prof, k_b,PROF_START, PROF_END)
    timepoints = prof.select_time_points(betas, 0.1,N=N_POINTS,  selection_mode="S2")
    traces_attack = scaled_traces[ATTACK_START:ATTACK_END]
    plaintexts_attack = plaintexts_all[ATTACK_START:ATTACK_END]

    if METHOD == "maximum":
        traces_prof = scaled_traces[total_prof:total_prof+N2]
        plaintexts_prof = plaintexts_all[total_prof:total_prof+N2]
        prof.estimate_covariance(traces_prof, plaintexts_prof, k_b)
    ke = KeyExtraction(prof)
    extracted_k, exp_keys = ke.extract_key(traces_attack, plaintexts_attack, betas, timepoints, TOTAL_ATTACK, method=METHOD)
    filtered = {k: v for k, v in exp_keys.items() if v}
    if key[byte_idx] in filtered.keys():
        correct_count += 1
    sorted_candidates = sorted(filtered.items(), key=lambda x: x[1], reverse=True)
    top_candidates = [int(k) for k, _ in sorted_candidates]

    correct_key = int(key[byte_idx])
    if correct_key==extracted_k:
        success = True
        correct_count += 1
    else:
        success = any((cand is not None and int(cand) == int(correct_key)) for cand in top_candidates)

    row = {
        "byte": byte_idx,
        "correct_key": correct_key,
        "extracted_key": int(extracted_k),
        "candidates": top_candidates,
        "success": success
    }
    results.append(row)

df_attack = pd.DataFrame(results,
                         columns=["byte", "correct_key", "extracted_key",
                                  "candidates","success"])

success_rate = (int(df_attack['success'].sum()) / 16) * 100
print(f"Overall Success Rate: {success_rate:.2f}%")

Overall Success Rate: 100.00%


# Known Key Attack using all possible combinations

In [ ]:
import time
import numpy as np
import pandas as pd
import itertools

ATTACK_START = 20000
ATTACK_END_LIST = [21000, 22000, 23000, 24000, 25000]
METHOD_LIST = ["minimum", "maximum"]
N_POINTS_LIST = [5, 10, 15, 20]
BASIS_TYPE_LIST = ["hw_bits", "hw"]
TOTAL_PROF_LIST = [5000, 10000]
key = np.array([15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0], dtype=np.uint8)
PROF_START = 0
PROF_END = 5000
N2 = 1000

AES_SBOX = [0x6, 0xB, 0x5, 0x4, 0x2, 0xE, 0x7, 0xA,
            0x9, 0xD, 0xF, 0xC, 0x3, 0x1, 0x0, 0x8]

all_results = []

for ATTACK_END, METHOD, N_POINTS, BASIS_TYPE, TOTAL_PROF in itertools.product(
        ATTACK_END_LIST, METHOD_LIST, N_POINTS_LIST, BASIS_TYPE_LIST, TOTAL_PROF_LIST):
    
    TOTAL_ATTACK = ATTACK_END - ATTACK_START
    results = []
    correct_count = 0

    # Lists for timing per byte
    prof_times_per_byte = []
    attack_times_per_byte = []

    for byte_idx in range(16):
        plaintexts_all = random_plaintexts[:, byte_idx] & 0x0F  # Lower 4 bits
        k_b = key[byte_idx] & 0x0F
        traces_prof = scaled_traces[:TOTAL_PROF]
        plaintexts_prof = plaintexts_all[:TOTAL_PROF]

        bf = BasisFunctions(AES_SBOX, basis_type=BASIS_TYPE)
        prof = Profiling(bf)

        # --- Profiling timing (includes estimate_betas, select_time_points, and covariance if used) ---
        t_prof_start = time.perf_counter()
        betas = prof.estimate_betas(traces_prof, plaintexts_prof, k_b, PROF_START, PROF_END)
        timepoints = prof.select_time_points(betas, 0.1, N=N_POINTS, selection_mode="S2")

        if METHOD == "maximum":
            # estimate_covariance is considered part of profiling
            traces_prof_ = scaled_traces[TOTAL_PROF:TOTAL_PROF+N2]
            plaintexts_prof_ = plaintexts_all[TOTAL_PROF:TOTAL_PROF+N2]
            prof.estimate_covariance(traces_prof_, plaintexts_prof_, k_b)

        t_prof_end = time.perf_counter()
        prof_time = t_prof_end - t_prof_start
        prof_times_per_byte.append(prof_time)

        # Prepare attack data
        traces_attack = scaled_traces[ATTACK_START:ATTACK_END]
        plaintexts_attack = plaintexts_all[ATTACK_START:ATTACK_END]

        # --- Attack timing (key extraction) ---
        ke = KeyExtraction(prof)
        t_attack_start = time.perf_counter()
        extracted_k, exp_keys = ke.extract_key(
            traces_attack, plaintexts_attack, betas, timepoints, TOTAL_ATTACK, method=METHOD
        )
        t_attack_end = time.perf_counter()
        attack_time = t_attack_end - t_attack_start
        attack_times_per_byte.append(attack_time)

        filtered = {k: v for k, v in exp_keys.items() if v}
        sorted_candidates = sorted(filtered.items(), key=lambda x: x[1], reverse=True)
        top_candidates = [int(k) for k, _ in sorted_candidates]
        correct_key = int(key[byte_idx])
        success = (correct_key == int(extracted_k)) or \
                  any((cand is not None and int(cand) == correct_key) for cand in top_candidates)

        results.append(success)

    success_rate = (sum(results) / 16) * 100

    # average times across 16 bytes for this parameter combination
    avg_prof_time = float(np.mean(prof_times_per_byte)) if prof_times_per_byte else 0.0
    avg_attack_time = float(np.mean(attack_times_per_byte)) if attack_times_per_byte else 0.0

    all_results.append({
        "ATTACK_END": ATTACK_END,
        "METHOD": METHOD,
        "N_POINTS": N_POINTS,
        "BASIS_TYPE": BASIS_TYPE,
        "Success_Rate(%)": round(success_rate, 2),
        "Avg_Profiling_Time_s": round(avg_prof_time, 6),
        "Avg_Attack_Time_s": round(avg_attack_time, 6)
    })

df_summary = pd.DataFrame(all_results)

# Print DataFrame
# sprint(df_summary.to_string(index=False))

# Print overall averages across ALL parameter combinations
overall_avg_prof = df_summary["Avg_Profiling_Time_s"].mean()
overall_avg_attack = df_summary["Avg_Attack_Time_s"].mean()
print("\nOverall averages across all parameter combinations:")
print(f"  Average profiling time per byte (s): {overall_avg_prof:.6f}")
print(f"  Average attack time per byte (s):    {overall_avg_attack:.6f}")


## Profiling and Key Extraction using Unknown Key

In [43]:
import time
import numpy as np
import pandas as pd

total_prof = 20000  # N1
N2 = 5000
ATTACK_START = 20000
ATTACK_END = 30000
TOTAL_ATTACK = ATTACK_END - ATTACK_START
PROF_START = 0
PROF_END = 5000
METHOD = "maximum"
N_POINTS = 10
BASIS_TYPE = "hw_bits"

try:
    from tqdm import tqdm
    use_tqdm = True
except Exception:
    use_tqdm = False

byte_iter = range(16)
if use_tqdm:
    byte_iter = tqdm(byte_iter, desc="bytes")

top_n = 5
AES_SBOX = [0x6, 0xB, 0x5, 0x4, 0x2, 0xE, 0x7, 0xA, 0x9, 0xD, 0xF, 0xC, 0x3, 0x1, 0x0, 0x8]
key = np.array([15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0], dtype=np.uint8)

best_candidates_per_byte = {}
candidate_range = range(16)
bf = BasisFunctions(AES_SBOX, basis_type=BASIS_TYPE)
prof = Profiling(bf)

# Timing accumulators
total_profiling_time = 0.0
total_attack_time = 0.0

for byte_idx in byte_iter:
    plaintexts_all = random_plaintexts[:, byte_idx] & 0x0F
    traces_prof = scaled_traces[:total_prof]
    plaintexts_prof = plaintexts_all[:total_prof]
    candidate_scores = []

    for cand in candidate_range:
        k_b = int(cand & 0x0F)

        # --- Profiling timing ---
        t_prof_start = time.perf_counter()
        try:
            betas = prof.estimate_betas(traces_prof, plaintexts_prof, k_b, PROF_START, PROF_END)
            timepoints = prof.select_time_points(betas, 0.1, N=N_POINTS, selection_mode="S2")

            if METHOD == "maximum":
                traces_prof_ = scaled_traces[total_prof:total_prof+N2]
                plaintexts_prof_ = plaintexts_all[total_prof:total_prof+N2]
                prof.estimate_covariance(traces_prof_, plaintexts_prof_, k_b)

        except Exception as e:
            print(f"[byte {byte_idx} cand {cand}] profiling failed: {e}")
            continue
        t_prof_end = time.perf_counter()
        total_profiling_time += (t_prof_end - t_prof_start)

        # --- Attack timing ---
        traces_attack = scaled_traces[ATTACK_START:ATTACK_END]
        plaintexts_attack = plaintexts_all[ATTACK_START:ATTACK_END]
        ke = KeyExtraction(prof)

        t_attack_start = time.perf_counter()
        try:
            extracted_k, exp_keys = ke.extract_key(
                traces_attack, plaintexts_attack, betas, timepoints, TOTAL_ATTACK, method=METHOD
            )
        except Exception as e:
            print(f"[byte {byte_idx} cand {cand}] extract_key failed: {e}")
            continue
        t_attack_end = time.perf_counter()
        total_attack_time += (t_attack_end - t_attack_start)

        # --- Scoring ---
        score = 0.0
        try:
            if isinstance(exp_keys, dict):
                score = float(exp_keys.get(cand, 0))
            else:
                score = 1.0 if extracted_k == cand else 0.0
        except Exception:
            score = 0.0

        if extracted_k == cand:
            score += 10.0

        try:
            exp_keys_snapshot = dict(exp_keys) if isinstance(exp_keys, dict) else {"value": exp_keys}
        except Exception:
            exp_keys_snapshot = {"unserializable": str(type(exp_keys))}

        candidate_scores.append((cand, score, extracted_k, exp_keys_snapshot))

    # Sort and store top candidates
    candidate_scores.sort(key=lambda x: x[1], reverse=True)
    best_candidates_per_byte[byte_idx] = candidate_scores[:top_n]

# --- Result Summary ---
top_k = 4
csv_filename = "byte_top4_separate_cols.csv"
all_success = True
success_count = 0
result_rows = []

for b in range(16):
    try:
        correct_key = int(key[b])
    except Exception:
        correct_key = None

    cand_list = best_candidates_per_byte.get(b, [])
    top_candidates = [int(t[0]) for t in cand_list[:top_k]]
    if len(top_candidates) < top_k:
        top_candidates += [None] * (top_k - len(top_candidates))

    if correct_key is None:
        success = False
    else:
        success = any((cand is not None and int(cand) == int(correct_key)) for cand in top_candidates)

    if success:
        success_count += 1
    else:
        all_success = False

    row = {
        "byte": b,
        "correct_key": correct_key,
        "cand_1": top_candidates[0],
        "cand_2": top_candidates[1],
        "cand_3": top_candidates[2],
        "cand_4": top_candidates[3],
        "success": success
    }
    result_rows.append(row)

df_top4 = pd.DataFrame(result_rows)
print(f"\n{total_prof} {TOTAL_ATTACK} {BASIS_TYPE} {N_POINTS} {METHOD}")

if all_success and success_count == 16:
    print("Key recovered successfully ✅")
else:
    print(f"Key recovery incomplete: {success_count}/16 bytes matched one of the top-{top_k} candidates.")

# --- Timing Summary ---
print("\n===== Timing Summary =====")
print(f"Total profiling time: {total_profiling_time:.3f} s")
print(f"Total attack time:    {total_attack_time:.3f} s")
print(f"Average profiling time per candidate: {total_profiling_time/(16*len(candidate_range)):.4f} s")
print(f"Average attack time per candidate:    {total_attack_time/(16*len(candidate_range)):.4f} s")


bytes: 100%|██████████| 16/16 [07:02<00:00, 26.39s/it]


20000 10000 hw_bits 10 maximum
Key recovery incomplete: 9/16 bytes matched one of the top-4 candidates.

===== Timing Summary =====
Total profiling time: 38.737 s
Total attack time:    383.549 s
Average profiling time per candidate: 0.1513 s
Average attack time per candidate:    1.4982 s
